In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
import requests
from tqdm.notebook import tqdm
import time
import random
from fake_useragent import UserAgent

In [85]:
def get_hist_values(link, sleep_time=0.6, random_sleep=True, random_agent=True):
    main_link = 'https://comicbookrealm.com' + link
    if random_agent:
        ua = UserAgent()
        headers = {'User-agent': f'{ua.random}'}
        
    else:
        headers = {'User-agent': ''}
        
    response = requests.get(main_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    dates = [td.text.strip() for td in soup.find_all('td', class_='item')]
    prices = [td.text.strip() for td in soup.find_all('td', class_='data')]
    
    frame = pd.DataFrame({'hist_prices_link': link, 'dates': str(dates), 
                          'prices': str(prices)}, index=[0])
    
    sleep(sleep_time + np.random.random()/2*random_sleep)
    
    return frame

def hist_scraper(data_path, data_range=(0, 10), sleep_time=0.6, random_sleep=True, random_agent=True):
    df = pd.read_parquet('all_data.parquet').iloc[data_range[0]: data_range[1]]
    frames = []
    iterator = tqdm(enumerate(df1.hist_prices_link))
    
    for i, link in iterator:
        if link is not None:
            iterator.set_description(f"Processing: {i + 1 + data_range[0]} / {df1.shape[0] + data_range[0]}")
            frame = get_hist_values(link, sleep_time, random_sleep, random_agent)
            frames.append(frame)
        else:
            continue
                                     
        if i%10000 == 0 and i > 0:
            temp = pd.concat(frames, axis=0)
            temp.to_csv(f'backup_data/hist_prices_data_{i + data_range[0]}.csv', index=False)
                                     
                            
    df_hist = pd.concat(frames, axis=0).reset_index(drop=True)
    df_hist.to_csv('hist_prices_data.csv', index=False)
    
    return df_hist

In [55]:
df = pd.read_parquet('all_data.parquet')
df = df.sort_values('current_value', ascending=False).reset_index(drop=True)

In [87]:
res = hist_scraper('all_data.parquet')

0it [00:00, ?it/s]

In [54]:
res

,dates,prices
0,"['Currently', 'November 3, 2015']","['$4.00', '$3.00']"
1,['Currently'],['$10.00']
2,"['Currently', 'March 6, 2013', 'September 10, ...","['$200.00', '$190.00', '$170.00', '$120.00']"
3,"['Currently', 'January 21, 2020']","['$6.00', '$5.00']"
4,['Currently'],['$3.00']
5,['Currently'],['$10.00']
6,['Currently'],['$14.00']
7,['Currently'],['$6.00']
8,['Currently'],['$8.00']
9,['Currently'],['$4.00']
